## ***Dự đoán hành vi mua hàng của người tiêu dùng***

### ML - NAIVE BAYES VỚI OVERSAMPLING (SMOTE)

## **I.** Khái quát về dự án:

### **1.1.** Bối cảnh:
Trong thương mại điện tử, việc hiểu hành vi người dùng giúp doanh nghiệp tối ưu quảng cáo, tăng tỷ lệ chuyển đổi và doanh thu. Dự án này sử dụng dữ liệu hành vi người dùng để dự đoán khả năng mua hàng.
### **1.2.** Mục tiêu:
- Xây dựng mô hình dự đoán xác suất người dùng mua hàng dựa trên các đặc điểm hành vi và sản phẩm.  
- Phân loại hành vi:
    - 0 → Không mua  
    - 1 → Mua hàng
### **1.3.** Vấn đề mất cân bằng dữ liệu:
Trong thực tế, số lượng người dùng thực hiện hành vi mua hàng (`purchase`) ít hơn rất nhiều so với chỉ xem (`view`). Điều này dẫn đến tập dữ liệu bị **mất cân bằng nghiêm trọng**, khiến mô hình có xu hướng dự đoán theo lớp đa số (không mua). Để giải quyết vấn đề này, chúng ta sẽ áp dụng kỹ thuật **Oversampling (SMOTE)**.

## **II.** Import thư viện và đọc dữ liệu

In [ ]:
import pandas as pd     
import numpy as np      
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split   
from sklearn.naive_bayes import GaussianNB             
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')  

pd.set_option('display.max_columns', 20)   
pd.set_option('display.width', 1000)       
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
data_path = r'D:\2019-Nov-100k.csv'
#data_path = '2019-Nov-100k.csv' # Thay đổi đường dẫn nếu cần
df = pd.read_csv(data_path, nrows=100000)
print(f"✓ Đã đọc dữ liệu thành công với {len(df):,} dòng và {df.shape[1]} cột.")
print("Các cột có trong tập dữ liệu: ")
print(df.columns.to_list())

## **III.** Khám phá dữ liệu ban đầu (EDA)

In [ ]:
print("Thông tin chi tiết về tập dữ liệu: ")
df.info()
print("\nThống kê mô tả cho các cột số: ")
print(df.describe())

In [ ]:
print("\n🛒 Thống kê các loại hành vi trong tập dữ liệu: ")
event_counts = df['event_type'].value_counts()
print(event_counts)

plt.figure(figsize=(8,6))
sns.barplot(x = event_counts.index, y = event_counts.values, palette='PuBuGn')
plt.title("Phân phối các loại hành vi")
plt.xlabel("Loại hành vi")
plt.ylabel("Số lượng")
plt.show()

## **IV.** Làm sạch và Tiền xử lý dữ liệu (Preprocessing)

In [ ]:
print("Thống kê giá sản phẩm: ")
display(df['price'].describe())
plt.boxplot(df['price'], vert=False)
plt.title("Phân phối giá sản phẩm")
plt.xlabel("Giá")
plt.show()

In [ ]:
# Xử lý giá trị thiếu
print("--- Xử lý giá trị thiếu ---")
df['price'] = df['price'].fillna(df['price'].median())
df['category_code'] = df['category_code'].fillna('unknown')
df['brand'] = df['brand'].fillna('unknown')
print("✓ Đã xử lý xong giá trị thiếu.")

# Chuyển đổi cột thời gian và tạo đặc trưng mới
print("\n--- Xử lý cột thời gian ---")
df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')
df['hour'] = df['event_time'].dt.hour
df['day'] = df['event_time'].dt.day
df['weekday'] = df['event_time'].dt.weekday
print("✓ Đã tạo các đặc trưng hour, day, weekday.")

# Xử lý trùng lặp
print("\n--- Xử lý trùng lặp ---")
duplicates_before = df.duplicated().sum()
df = df.drop_duplicates()
print(f"✓ Đã loại bỏ {duplicates_before} dòng trùng lặp.")

# Xử lý giá trị ngoại lai (outliers) trong cột 'price'
print("\n--- Xử lý giá trị ngoại lai ---")
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers_count = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)].shape[0]
df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
print(f"✓ Đã loại bỏ {outliers_count} giá trị ngoại lai trong cột 'price'.")
print(f"\nKích thước dữ liệu sau khi làm sạch: {df.shape}")

## **V.** Chuẩn bị dữ liệu và chia Train-Test

Đây là bước quan trọng nhất để tránh rò rỉ dữ liệu. Chúng ta sẽ chia dữ liệu thô (sau khi làm sạch cơ bản) thành các tập train và test trước khi thực hiện bất kỳ bước mã hóa hoặc chuẩn hóa nào.

In [ ]:
# Tạo biến mục tiêu
# Lưu ý: Chúng ta tạo cột 'is_purchase' từ 'event_type', 
# do đó 'event_type' không được dùng làm feature nữa.
df['is_purchase'] = (df['event_type'] == 'purchase').astype(int)

# Loại bỏ 'event_type' khỏi danh sách features để tránh rò rỉ dữ liệu
features = ['category_code', 'brand', 'price', 'hour', 'day', 'weekday']

X = df[features]
y = df['is_purchase']

# Chia dữ liệu TRƯỚC KHI xử lý
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kích thước tập huấn luyện (X_train): {X_train.shape}")
print(f"Kích thước tập kiểm thử (X_test):  {X_test.shape}")

## **VI.** Mã hóa và Chuẩn hóa dữ liệu (Theo đúng quy trình)

In [ ]:
# Xác định các cột cần xử lý
categorical_cols = ['category_code', 'brand']
numerical_cols = ['price', 'hour', 'day', 'weekday']

# Tạo bản sao để tránh SettingWithCopyWarning
X_train = X_train.copy()
X_test = X_test.copy()

# 1. Mã hóa các cột categorical
for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    # Xử lý các giá trị chưa từng thấy trong tập test
    X_test[col] = X_test[col].map(lambda s: -1 if s not in le.classes_ else le.transform([s])[0])

# 2. Chuẩn hóa các cột số
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("✓ Dữ liệu đã được mã hóa và chuẩn hóa theo đúng quy trình.")

## **VII.** Cân bằng dữ liệu huấn luyện với Oversampling (SMOTE)

In [ ]:
print("Phân bố lớp TRƯỚC khi áp dụng SMOTE:")
print(y_train.value_counts())

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("\nPhân bố lớp SAU khi áp dụng SMOTE:")
print(y_train_smote.value_counts())

## **VIII.** Huấn luyện mô hình Gaussian Naive Bayes

###  8.1. Kiến thức nền tảng về Naive Bayes

**Naive Bayes** là một nhóm các thuật toán phân loại dựa trên **Định lý Bayes** với một giả định "ngây thơ" (naive) về sự độc lập có điều kiện giữa các đặc trưng.

**Công thức Bayes cơ bản:**
$$P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)}$$

Trong đó:
- $Y$: là biến mục tiêu (lớp cần dự đoán, ví dụ: `mua` hoặc `không mua`).
- $X$: là các đặc trưng đầu vào (ví dụ: `price`, `hour`, `brand`).
- $P(Y|X)$: là xác suất có điều kiện của lớp $Y$ khi biết các đặc trưng $X$ (đây là xác suất chúng ta muốn tìm).
- $P(X|Y)$: là xác suất xuất hiện các đặc trưng $X$ khi biết lớp là $Y$.

**Giả định "ngây thơ"**: Thuật toán giả định rằng tất cả các đặc trưng là độc lập với nhau khi đã biết lớp. Điều này giúp đơn giản hóa việc tính toán $P(X|Y)$ bằng cách nhân xác suất của từng đặc trưng riêng lẻ.

**Gaussian Naive Bayes**:
Trong trường hợp này, chúng ta sử dụng **Gaussian Naive Bayes** vì các đặc trưng số của chúng ta (như `price`, `hour` sau khi chuẩn hóa) có thể được giả định là tuân theo **phân phối chuẩn (Gaussian)**. Thuật toán sẽ tính toán giá trị trung bình (mean) và phương sai (variance) cho từng đặc trưng ứng với mỗi lớp để ước tính xác suất.
$$P(x_i | y) = \frac{1}{\sqrt{2\pi\sigma_y^2}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma_y^2}\right)$$

In [ ]:
# Huấn luyện mô hình trên dữ liệu đã được cân bằng bằng SMOTE
model_nb = GaussianNB()
model_nb.fit(X_train_smote, y_train_smote)

print("✅ Mô hình Gaussian Naive Bayes đã được huấn luyện thành công.")

## **IX.** Đánh giá mô hình

In [ ]:
# Đánh giá trên tập test gốc để phản ánh hiệu năng thực tế
y_pred = model_nb.predict(X_test)

# 1. In báo cáo phân loại chi tiết cho mô hình Naive Bayes
print("📊 Báo cáo phân loại chi tiết cho mô hình Naive Bayes:\n")
print(classification_report(y_test, y_pred))

### **So sánh với mô hình dự đoán ngẫu nhiên**

Để so sánh một cách công bằng, chúng ta tạo ra một mô hình dự đoán ngẫu nhiên nhưng có cùng **tỷ lệ dự đoán "mua"** như mô hình Naive Bayes. Điều này giúp kiểm tra xem mô hình của chúng ta có thực sự "thông minh" hơn việc đoán ngẫu nhiên với cùng một tần suất hay không.

In [ ]:
# Tính tỷ lệ dự đoán mua hàng của mô hình Naive Bayes
nb_prediction_rate = np.mean(y_pred)
print(f"Tỷ lệ dự đoán 'Mua' của mô hình Naive Bayes: {nb_prediction_rate:.4f}")

# Tạo dự đoán ngẫu nhiên với cùng tỷ lệ trên
np.random.seed(42) # Để đảm bảo kết quả có thể tái lập
y_pred_random = np.random.choice([0, 1], size=len(y_test), p=[1 - nb_prediction_rate, nb_prediction_rate])

print("\n📊 Báo cáo phân loại cho mô hình dự đoán ngẫu nhiên:\n")
print(classification_report(y_test, y_pred_random))

# Lấy ma trận nhầm lẫn cho cả hai mô hình
cm_nb = confusion_matrix(y_test, y_pred)
cm_random = confusion_matrix(y_test, y_pred_random)

# Vẽ hai ma trận nhầm lẫn cạnh nhau để so sánh
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Ma trận của Naive Bayes
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0],
            xticklabels=['Không mua', 'Mua'], yticklabels=['Không mua', 'Mua'])
axes[0].set_title("🔍 Confusion Matrix - Naive Bayes")
axes[0].set_xlabel("Dự đoán")
axes[0].set_ylabel("Thực tế")

# Ma trận của mô hình ngẫu nhiên
sns.heatmap(cm_random, annot=True, fmt='d', cmap='Greens', cbar=False, ax=axes[1],
            xticklabels=['Không mua', 'Mua'], yticklabels=['Không mua', 'Mua'])
axes[1].set_title("🎲 Confusion Matrix - Dự đoán ngẫu nhiên")
axes[1].set_xlabel("Dự đoán")
axes[1].set_ylabel("Thực tế")

plt.suptitle('So sánh hiệu suất mô hình', fontsize=16)
plt.show()

## **X.** Tổng kết

### **Tóm tắt kết quả:**
1.  **Vấn đề ban đầu:** Dữ liệu bị mất cân bằng nghiêm trọng với lớp 'mua hàng' (lớp 1) chỉ chiếm ~1.4%. Nếu không xử lý, mô hình sẽ có xu hướng bỏ qua lớp này.

2.  **Giải pháp:** Chúng ta đã áp dụng thành công kỹ thuật **SMOTE (Oversampling)** trên tập huấn luyện để tạo ra các mẫu tổng hợp, giúp cân bằng số lượng giữa hai lớp.

3.  **Kết quả của mô hình Naive Bayes:**
    *   **Recall (Lớp 1 - Mua hàng):** Chỉ số này đã tăng lên một mức đáng kể, cho thấy mô hình đã có khả năng **phát hiện được một phần** các trường hợp khách hàng thực sự mua hàng, thay vì bỏ sót hoàn toàn.
    *   **Precision (Lớp 1 - Mua hàng):** Chỉ số này không cao, cho thấy mô hình vẫn còn dự đoán sai khá nhiều (dự đoán "mua" nhưng thực tế là "không mua"). Đây là sự đánh đổi thường thấy để có được Recall cao hơn.
    *   **So sánh Confusion Matrix:** Nhìn vào hai ma trận nhầm lẫn, ta thấy rõ sự khác biệt. Mặc dù cả hai mô hình có thể dự đoán "mua" với số lượng tương đương (tổng cột 'Mua' gần bằng nhau), nhưng mô hình Naive Bayes phân bổ các dự đoán đó một cách "thông minh" hơn nhiều. Cụ thể, số lượng **True Positives** (dự đoán đúng là 'Mua') của Naive Bayes cao hơn đáng kể so với mô hình ngẫu nhiên, trong khi số lượng **False Positives** (dự đoán sai là 'Mua') lại thấp hơn.

### **Kết luận:**
Việc áp dụng **Oversampling (SMOTE)** đã chứng tỏ hiệu quả trong việc giải quyết vấn đề mất cân bằng dữ liệu. Mô hình Naive Bayes, sau khi được huấn luyện trên dữ liệu cân bằng, đã trở nên hữu ích hơn trong thực tế vì nó có khả năng xác định các khách hàng tiềm năng, dù phải đánh đổi một chút về độ chính xác của các dự đoán này.

**Việc so sánh trực tiếp với một mô hình dự đoán ngẫu nhiên có cùng tỷ lệ đầu ra** đã khẳng định một cách mạnh mẽ rằng mô hình Naive Bayes đã thực sự học được các mẫu (patterns) có ý nghĩa từ dữ liệu. Nó không chỉ đơn thuần đưa ra dự đoán mà còn đưa ra những dự đoán có độ chính xác cao hơn hẳn so với việc tung đồng xu. Điều này chứng minh giá trị của việc xây dựng mô hình học máy trong bài toán này.